### 3.1 处理缺失数据
#### 1. 缺失值的检测,删除,填充
1. 检测缺失值: 数据结构.isnull() , 返回DataFrame或Series
2. 什么值被识别成缺失值 :   
  1. python的None对象
  2. np.nan对象
3. Series.dropna : 返回不带缺失值的Series  
 DataFrame.dropna(how='all/any' , axis=0/1) : 删除整行/整列, 根据全部缺失或部分缺失
4. 填充缺失值:  
 数据结构.fillna(method='',values='')  
  1. mathod: 填充策略  
    1. ffill : 使用上一个值填充
    2. bfill : 使用下一个值填充
    3. None : 不适用策略,自己指定values指
  2. value : 自定义填充值  
    1. value可以为scalar, 字典, Series, DataFrame
    2. value=字典: (key=columnIdx,value=填充值), 指定每一列的填充值
  


In [19]:
import pandas as pd
from pandas import Series,DataFrame
import numpy as np

df = pd.DataFrame([[np.nan, 2, np.nan, 0],[3, 4, np.nan, 1], [np.nan, np.nan, np.nan, 5],[np.nan, 3, np.nan, 4]],columns=list('ABCD'))
obj = Series([1,2,3,np.nan,5,6])
print df.isnull()
print obj.isnull()
print "=============[1]============="
# 删除缺失值
print obj.dropna()
print df.dropna(how='all')
print "=============[2]============="
# 填充缺失值
print df.fillna(value={'A':-1,'B':-2,'C':-3})
print df.fillna(method='ffill')

       A      B     C      D
0   True  False  True  False
1  False  False  True  False
2   True   True  True  False
3   True  False  True  False
0    False
1    False
2    False
3     True
4    False
5    False
dtype: bool
=============[1]=============
0    1.0
1    2.0
2    3.0
4    5.0
5    6.0
dtype: float64
     A    B   C  D
0  NaN  2.0 NaN  0
1  3.0  4.0 NaN  1
2  NaN  NaN NaN  5
3  NaN  3.0 NaN  4
=============[2]=============
     A    B    C  D
0 -1.0  2.0 -3.0  0
1  3.0  4.0 -3.0  1
2 -1.0 -2.0 -3.0  5
3 -1.0  3.0 -3.0  4
     A    B   C  D
0  NaN  2.0 NaN  0
1  3.0  4.0 NaN  1
2  3.0  4.0 NaN  5
3  3.0  3.0 NaN  4


### 3.2 数据转换
#### 1. 重复行的处理
1. 检测重复行:   
 DataFrame.duplicated() , 返回Boolean的Series(表示该行是否为重复行)
2. 删除重复行  
  1. DataFrame.drop_duplicates()
  2. DataFrame.drop_duplicates([columnIdx]) : 指定列上若有重复值, 则删除整行  
  [注] : 删除重复行, 默认保留最先出现的行. 可使用keep='last'保留最后出现的行


In [21]:
df = DataFrame({'k1':['one','two']*3+['two'], 'k2':[1,1,2,3,3,4,4]})
df

,k1,k2
0,one,1
1,two,1
2,one,2
3,two,3
4,one,3
5,two,4
6,two,4


In [25]:
# 检测重复行
print df.duplicated()
print "=============[1]============="
# 删除重复行
print df.drop_duplicates()
# 删除指定列上有重复值的行
print df.drop_duplicates(['k1'])

0    False
1    False
2    False
3    False
4    False
5    False
6     True
dtype: bool
=============[1]=============
    k1  k2
0  one   1
1  two   1
2  one   2
3  two   3
4  one   3
5  two   4
    k1  k2
0  one   1
1  two   1


#### 2. 值映射
1. Series.map(dict/lambda func) :  
 将一个Series映射成另一个Series. 若用dict指定, 则是吧Series中的元素匹配key, 转换成对应的value  
 若传入lambda函数 : 把lambda函数应用在Series的每个元素上
 2. DataFrame的值映射  
 由于map方法只能应用在Series上, 而DataFrame能通过apply(f)应用在DataFrame的每列Series上, 所以组合出了applymap(f)方法  
 该方法可以讲func分别应用在DataFrame的每列上

In [37]:
df = pd.DataFrame({'food': ['bacon', 'pulled pork', 'bacon','Pastrami', 'corned beef', 'Bacon', 'pastrami', 'honey ham', 'nova lox'],
                                'ounces': [4, 3, 12, 6, 7.5, 8, 3, 5, 6]})
df

,food,ounces
0,bacon,4.0
1,pulled pork,3.0
2,bacon,12.0
3,Pastrami,6.0
4,corned beef,7.5
5,Bacon,8.0
6,pastrami,3.0
7,honey ham,5.0
8,nova lox,6.0


In [42]:
# Series的值映射map()
mydict = {
  'bacon': 'pig',
  'pulled pork': 'pig',
  'pastrami': 'cow',
  'corned beef': 'cow',
  'honey ham': 'pig',
  'nova lox': 'salmon'
}
print df['food'].str.lower().map(mydict) # .str提供字符串转换方法
f = lambda x:mydict[x.lower()]
f2 = lambda x:x*2
print df['food'].map(f)
print "=============[1]============="
# DataFrame的值映射
df.applymap(f2)
# DataFrame删除全0行 : 先用applymap把0映射成None,再用dropna(how='all')删除
df2 = DataFrame({'a':[1,2,0],'b':[9,9,0]})
zerofunc = lambda x: None if (x==0) else x 
df2 = df2.applymap(zerofunc)
df2.dropna(how='all')

0       pig
1       pig
2       pig
3       cow
4       cow
5       pig
6       cow
7       pig
8    salmon
Name: food, dtype: object
0       pig
1       pig
2       pig
3       cow
4       cow
5       pig
6       cow
7       pig
8    salmon
Name: food, dtype: object
=============[1]=============


,a,b
0,1.0,9.0
1,2.0,9.0


#### 3. 值替换
1. 数据结构.fillna可看做值替换的特殊形式: 把NaN替换成指定值  
 Series.map()也是替换数据的子集, replace()提供了更灵活的办法
2. 数据结构.replace(src,dist) : 把src替换成dist  
  1. method参数: 替换策略, 默认为None. 当dist=None时, 按照替换策略替换值.相当于fillna(method=''). 参数选择同样有ffill, backfill  
  2. replace(dict) : 将key作为src, valjue作为dist

In [52]:
 obj = pd.Series([1., -999., 2., -999., -1000., 3.])
print obj
# 将-999替换成NaN
print obj.replace(-999,np.nan)
print obj.replace([-999,-1000],np.nan)
print ("=============[1]============")
# 替换参照dict
replaceDict = {-999:np.nan,-1000:np.nan}
print obj.replace(replaceDict)


0       1.0
1    -999.0
2       2.0
3    -999.0
4   -1000.0
5       3.0
dtype: float64
0       1.0
1       NaN
2       2.0
3       NaN
4   -1000.0
5       3.0
dtype: float64
0    1.0
1    NaN
2    2.0
3    NaN
4    NaN
5    3.0
dtype: float64
=============[1]============


0    1.0
1    NaN
2    2.0
3    NaN
4    NaN
5    3.0
dtype: float64

In [55]:
df.replace(mydict)

,food,ounces
0,pig,4.0
1,pig,3.0
2,pig,12.0
3,Pastrami,6.0
4,cow,7.5
5,Bacon,8.0
6,cow,3.0
7,pig,5.0
8,salmon,6.0


#### 4. index重命名
1. Index.map(func), 使用函数替换索引名称
2. DataFrame.rename(index=func,columns=func) : 替换rowidx和columnIdx的方法

In [60]:
df = pd.DataFrame(np.arange(12).reshape((3, 4)), index=['Ohio', 'Colorado', 'New York'], columns=['one', 'two', 'three', 'four'])
df

,one,two,three,four
Ohio,0,1,2,3
Colorado,4,5,6,7
New York,8,9,10,11


In [61]:
df.index = data.index.map(lambda x:x.upper())
df

,one,two,three,four
OHIO,0,1,2,3
COLORADO,4,5,6,7
NEW YORK,8,9,10,11


In [67]:
# Series重命名索引
print df['one'].rename(str.lower)
# DataFrame重命名索引
df.rename(index=str.title,columns=str.lower)

ohio        0
colorado    4
new york    8
Name: one, dtype: int64


,one,two,three,four
Ohio,0,1,2,3
Colorado,4,5,6,7
New York,8,9,10,11
